In [2]:
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt


import sys
sys.path.append("../")
from ccfm.ccfm import (
    load_cfm_traces, 
    make_3d_fault_mesh, 
    load_nshm_traces, 
    load_canada_traces,
    make_tri_mesh,
    write_cfm_tri_meshes,
    write_cfm_trace_geojson,
)

from ccfm.geom import (
    sample_polyline,
    sample_polyline_to_n_pts,
    add_fixed_elev_to_trace,
    _draw_pt_profile,
    get_contours_from_profiles,
    haversine_distance,
    _straight_profile_n_pts,
    EARTH_RAD_KM,
    get_values_at_coordinates,
    get_resampled_trace_elevations,
)

from ccfm.cfm_io import make_3d_tri_multipolygon, write_json

import json

In [3]:
yfp_contour_file = "../misc_data/cross_sections/yakima_folds/yakima_fault_contours.geojson"

with open(yfp_contour_file) as f:
    ygj = json.load(f)
    
    contours = {}
    for ff in ygj['features']:
        contours[ff['properties']['name']] = ff

In [4]:
man_contours = [
    contours['manastash_w_trace'],
    contours['manastash_w_1'],
    contours['manastash_w_2'],
    contours['manastash-767'],
    contours['manastash-1612'],
    contours['manastash-2201'],
    contours['manastash-2289'],
    contours['manastash-2864'],
    contours['manastash-3159'],
    contours['manastash-3360'],
    contours['manastash-3709'],
    contours['manastash-4100'],
    contours['manastash-4634'],
    contours['manastash-4894'],
    contours['manastash-9053'],
]

In [5]:
def prepare_fault_contours(fault_contours, **kwargs):
    #fc_sorted = sorted(fault_contours, key=lambda x: x['properties']['level'])
    fc_sorted = fault_contours
    trace = fc_sorted[0]
    trace_sampled = sample_polyline(trace['geometry']['coordinates'], **kwargs)
    trace_sampled = add_fixed_elev_to_trace(trace_sampled, trace['properties']['elev'])
    
    contours_out = []
    contours_out.append(trace_sampled)
    
    n_trace_pts = len(trace_sampled)
    for trace in fc_sorted[1:]:
        trace_sampled = sample_polyline_to_n_pts(trace['geometry']['coordinates'], n_trace_pts)
        trace_sampled = add_fixed_elev_to_trace(trace_sampled, trace['properties']['elev'])
        contours_out.append(trace_sampled)
    
    
    return contours_out

In [6]:
prepped_contours = prepare_fault_contours(man_contours, pt_distance=0.5)

In [7]:
prepped_contours[0]

[[-120.89737255321434, 47.00394668470463, 99999.0],
 [-120.88868051077874, 47.00003032914189, 99999.0],
 [-120.8794868162351, 46.996739511385094, 99999.0],
 [-120.86922090879504, 46.99554350398029, 99999.0],
 [-120.858899079289, 46.994608594290966, 99999.0],
 [-120.84999072681913, 46.991096209657876, 99999.0],
 [-120.84226807658952, 46.986523245041795, 99999.0],
 [-120.8332023083075, 46.9831631078073, 99999.0],
 [-120.82532729660785, 46.978528814431556, 99999.0],
 [-120.81789592292643, 46.97356031985565, 99999.0],
 [-120.81087514628314, 46.968345668863144, 99999.0],
 [-120.80306641188292, 46.96370205129712, 99999.0],
 [-120.79362990508517, 46.96073109541091, 99999.0],
 [-120.78383868968207, 46.95845836152079, 99999.0],
 [-120.77343930116226, 46.95817428391393, 99999.0],
 [-120.76303313496649, 46.95823449319486, 99999.0],
 [-120.75263382845742, 46.958495327246865, 99999.0],
 [-120.74243969038766, 46.95964476622231, 99999.0],
 [-120.73262387413025, 46.96185838734606, 99999.0],
 [-120.722

In [11]:
manastash_w_trace_elevs = get_values_at_coordinates("/Users/itchy/gis/dems/srtm/pnw_30m_wgs84.tif",
                          [c[:2] for c in prepped_contours[0]])

In [12]:
for i, c in enumerate(prepped_contours[0]):
    c[2] = manastash_w_trace_elevs[i]

In [13]:
prepped_contours[0]

[[-120.89737255321434, 47.00394668470463, 1244],
 [-120.88868051077874, 47.00003032914189, 1285],
 [-120.8794868162351, 46.996739511385094, 1207],
 [-120.86922090879504, 46.99554350398029, 1096],
 [-120.858899079289, 46.994608594290966, 1009],
 [-120.84999072681913, 46.991096209657876, 1005],
 [-120.84226807658952, 46.986523245041795, 981],
 [-120.8332023083075, 46.9831631078073, 943],
 [-120.82532729660785, 46.978528814431556, 928],
 [-120.81789592292643, 46.97356031985565, 977],
 [-120.81087514628314, 46.968345668863144, 945],
 [-120.80306641188292, 46.96370205129712, 898],
 [-120.79362990508517, 46.96073109541091, 830],
 [-120.78383868968207, 46.95845836152079, 872],
 [-120.77343930116226, 46.95817428391393, 896],
 [-120.76303313496649, 46.95823449319486, 908],
 [-120.75263382845742, 46.958495327246865, 879],
 [-120.74243969038766, 46.95964476622231, 915],
 [-120.73262387413025, 46.96185838734606, 852],
 [-120.72223615436044, 46.96185049005157, 863],
 [-120.71184477490183, 46.961632

In [14]:
def make_mesh_from_prepared_contours(contours, vert_pt_spacing=None, down_dip_pt_spacing=None):

    num_contour_sets = len(contours) - 1
    
    all_contours = []
    
    for i_cs in range(num_contour_sets):
        top_contour_elev = contours[i_cs][0][2]
        bottom_contour_elev = contours[i_cs+1][0][2]
        vert_distance = (top_contour_elev - bottom_contour_elev) / 1000.
        
        # get horizontal distance between points
        # using first point
        hor_distance = haversine_distance(contours[i_cs][0][0],
                                          contours[i_cs][0][1],
                                          contours[i_cs+1][0][0],
                                          contours[i_cs+1][0][1],
                                         )
        
        down_dip_distance = np.sqrt(vert_distance**2 + hor_distance**2)
        
        n_pts = int(round( down_dip_distance / down_dip_pt_spacing)) + 1
          
        profiles = [_draw_pt_profile(contours[i_cs][j],
                                     contours[i_cs+1][j],
                                     n_pts)
                    for j in range(len(contours[i_cs]))]
        
        
        if i_cs == 0:
            return_top = True
        else:
            return_top = False
        
        contour_set = get_contours_from_profiles(profiles,
                                                return_top,
                                                )
        all_contours.extend(contour_set)
        
    return all_contours

In [17]:
mesh = make_mesh_from_prepared_contours(prepped_contours,
                                            down_dip_pt_spacing=0.5)
tri_mesh = make_tri_mesh(mesh)

In [18]:
write_cfm_tri_meshes("../misc_data/cross_sections/yakima_folds/manastash_3d.geojson",
                     [tri_mesh],
                     [{'properties':{'name': 'manastash_3d'}}],
                    )